# unsloth 微调
基于unsloth官方教程

## 安装unsloth
```bash
pip install unsloth
```

## 加载模型

In [13]:
from unsloth import FastLanguageModel
import torch

MODEL_PATH='/root/autodl-tmp/Qwen3'
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_PATH,
    max_seq_length=512,
    load_in_4bit=True,
    dtype=None,
    local_files_only=True,
    trust_remote_code=True,
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA vGPU-32GB. Num GPUs = 1. Max memory: 31.473 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## 添加LoRA适配器

In [14]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024, padding_idx=151643)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLUActivation()
    

In [15]:
model = FastLanguageModel.get_peft_model(
    model,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    r=8, 
    lora_alpha=16,
    lora_dropout=0.05,
    bias='none',
    use_gradient_checkpointing='unsloth',
    random_state=2025,
    use_rslora=False,
    loftq_config=None
)

In [16]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024, padding_idx=151643)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

## 数据准备
仍然使用医疗问答数据集

In [18]:
# 转换数据格式
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template='qwen3-instruct'
)

In [26]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='./autodl-tmp/data/train_zh_1000.jsonl')['train']
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 1000
})


In [27]:
dataset[100]

{'instruction': '流涕不止怎么办，无',
 'input': '',
 'output': '多考虑是属于鼻炎的情况，多由于受凉或感冒迁延不愈引起的情况。建议到五官科检查一下实际的情况为宜，在明确为鼻炎的情况后建议在医生的指导下口服敏迪 香菊胶囊 新康泰克做调节治疗'}

In [24]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

In [25]:
dataset[100]

{'instruction': '流涕不止怎么办，无',
 'input': '',
 'output': '多考虑是属于鼻炎的情况，多由于受凉或感冒迁延不愈引起的情况。建议到五官科检查一下实际的情况为宜，在明确为鼻炎的情况后建议在医生的指导下口服敏迪 香菊胶囊 新康泰克做调节治疗'}

In [32]:
qwen3_prompt = """<|im_start|>user
{}<|im_end|>
<|im_start|>assistant
{}<|im_end|>
"""
def formatting_prompt_func(examples):
    instructions = examples['instruction']
    outputs = examples['output']
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = qwen3_prompt.format(instruction, output)
        texts.append(text)
    return {'text': texts}

In [33]:
dataset = dataset.map(formatting_prompt_func, batched=True)

dataset[100]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'instruction': '流涕不止怎么办，无',
 'input': '',
 'output': '多考虑是属于鼻炎的情况，多由于受凉或感冒迁延不愈引起的情况。建议到五官科检查一下实际的情况为宜，在明确为鼻炎的情况后建议在医生的指导下口服敏迪 香菊胶囊 新康泰克做调节治疗',
 'text': '<|im_start|>user\n流涕不止怎么办，无<|im_end|>\n<|im_start|>assistant\n多考虑是属于鼻炎的情况，多由于受凉或感冒迁延不愈引起的情况。建议到五官科检查一下实际的情况为宜，在明确为鼻炎的情况后建议在医生的指导下口服敏迪 香菊胶囊 新康泰克做调节治疗<|im_end|>\n'}

## 微调模型训练

In [34]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    )
)

Unsloth: Tokenizing ["text"] (num_proc=64):   0%|          | 0/1000 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [35]:
# 希望损失函数值计算回答部分
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=64):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [38]:
# before masking
tokenizer.decode(trainer.train_dataset[100]['input_ids'])

'<|im_start|>user\n流涕不止怎么办，无<|im_end|>\n<|im_start|>assistant\n多考虑是属于鼻炎的情况，多由于受凉或感冒迁延不愈引起的情况。建议到五官科检查一下实际的情况为宜，在明确为鼻炎的情况后建议在医生的指导下口服敏迪 香菊胶囊 新康泰克做调节治疗<|im_end|>\n'

In [46]:
# after masking
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'              多考虑是属于鼻炎的情况，多由于受凉或感冒迁延不愈引起的情况。建议到五官科检查一下实际的情况为宜，在明确为鼻炎的情况后建议在医生的指导下口服敏迪 香菊胶囊 新康泰克做调节治疗<|im_end|>\n'

In [47]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 5,046,272 of 601,096,192 (0.84% trained)


Step,Training Loss
1,3.668200
2,3.864400
3,4.201600
4,5.128900
5,4.198500
6,3.522900
7,3.617200
8,3.656600
9,3.354400
10,3.208400


TrainOutput(global_step=60, training_loss=3.327336545785268, metrics={'train_runtime': 106.3355, 'train_samples_per_second': 4.514, 'train_steps_per_second': 0.564, 'total_flos': 259272951201792.0, 'train_loss': 3.327336545785268, 'epoch': 0.48})

## 推理

In [54]:
message = [
    {"role" : "user", "content" : "癌症的特征是什么？"}
]
text = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=True
)
text

'<|im_start|>user\n癌症的特征是什么？<|im_end|>\n<|im_start|>assistant\n'

In [56]:
from transformers import TextStreamer
input = tokenizer(text, return_tensors='pt').to('cuda')
model.generate(
    **input,
    max_new_tokens=256,
    temperature = 0.7, top_p = 0.8, top_k = 20,
    streamer=TextStreamer(tokenizer, skip_prompt = True)
)

恶性程度高，发生率高，病程长，预后差，晚期高，早期低，治疗效果差，晚期低，早期高，治疗效果差，晚期高，早期低，晚期高，早期低<|im_end|>


tensor([[151644,    872,    198, 107397,   9370, 104363, 102021,  11319, 151645,
            198, 151644,  77091,    198, 114210, 100069,  44636,   3837,  99726,
          95355,  44636,   3837,  99252,  38507,  45861,   3837,  98841,  33447,
          99572,   3837, 113040,  44636,   3837, 105184,  99285,   3837, 101899,
         101062,  99572,   3837, 113040,  99285,   3837, 105184,  44636,   3837,
         101899, 101062,  99572,   3837, 113040,  44636,   3837, 105184,  99285,
           3837, 113040,  44636,   3837, 105184,  99285, 151645]],
       device='cuda:0')

效果并不是很好，可能和模型大小和数据集大小有关

## 保存模型

In [ ]:
model.save_pretrained('lora_model')
tokenizer.save_pretrained('lora_model')